In [1]:
import pandas as pd
import json
import pickle

In [2]:
d_path = 'dataset'

In [3]:
with open('../exports/%s/json_format_balanced_combined/test-0-1814.txt' % (d_path), 'rb') as f:
    test = json.load(f)
    
with open('../exports/%s/json_format_balanced_combined/train-0-7787.txt' % (d_path), 'rb') as f:
    train = json.load(f)

In [4]:
total = {}
total.update(train)
total.update(test)

In [5]:
test_compo = {}
test_steps = {}
for i in test:
    q = test[i]
    if q['novel_comp'] == 1:
        test_compo[i] = q
    if q['more_steps'] == 1:
        test_steps[i] = q
        
train_compo = {}
train_steps = {}
for i in train:
    q = train[i]
    if q['novel_comp'] == 0:
        train_compo[i] = q
    if q['more_steps'] == 0:
        train_steps[i] = q
        
total_compo = {}
total_compo.update(train_compo)
total_compo.update(test_compo)

total_steps = {}
total_steps.update(train_steps)
total_steps.update(test_steps)

- [x] All balanced ids match json ids and vice versa
- [x] Everything that is marked compo/steps is in compo/steps and vice versa
- [x] Switched to lowercase if do that
- [x] No duplicated keys
- [x] They are the same size (and the same size as the published numbers in the pdf)
- it mat


# TODO: check that total is test + val + train

In [51]:
def getCSVfinal_csv(x):
    train = pd.read_csv('../exports/%s/final_csvs/%s-%s.csv' % (d_path, 'train', x))
    val = pd.read_csv('../exports/%s/final_csvs/%s-%s.csv' % (d_path, 'valid', x))
    train = pd.concat([train, val])
    test = pd.read_csv('../exports/%s/final_csvs/%s-%s.csv' % (d_path, 'test', x))
    total = pd.read_csv('../exports/%s/final_csvs/%s-%s.csv' % (d_path, 'total', x))
    
    return train, test, total


def getCSV(x):
    if x == 'balanced':
        folder = 'balanced'
    elif x == 'compo':
        folder = 'novel compositions'
    elif x == 'steps':
        x = 'steps_templ'
        folder = 'more compositional steps'
    train = pd.read_csv('../ALL FINAL DATA/agqa_2.0/csvs/%s/Train_frameqa_question-%s.csv' % (folder, x))
    test = pd.read_csv('../ALL FINAL DATA/agqa_2.0/csvs/%s/Test_frameqa_question-%s.csv' % (folder, x))
    total = pd.read_csv('../ALL FINAL DATA/agqa_2.0/csvs/%s/Total_frameqa_question-%s.csv' % (folder, x))

    return train, test, total


def getTXT(x):
    if x == 'balanced':
        return train, test, total
    elif x == 'compo':
        return train_compo, test_compo, total_compo
    elif x == 'steps':
        return train_steps, test_steps, total_steps
    else:
        print('invalid type %s' % x)

In [52]:
def sameSize(csv, txt):
    return csv.shape[0] == len(txt)

In [53]:
def keysUnique(csv):
    keys = list(csv.key.values)
    return len(keys) == len(set(keys))

In [54]:
def allLower(csv, txt):
    answers = set(csv['answer'].values)
    for i in answers:
        if i != i.lower():
            print("CSV Uppercase answer: %s" % i)
            return False
    
    answers = set([txt[i]['answer'] for i in txt])
    for ans in answers:
        if ans != ans.lower():
            print("TXT Uppercase answer: %s" % i)
            return False
    
    return True

In [55]:
def keysMatch(csv, txt):
    csv_keys = list(csv['key'].values)
    txt_keys = list(txt.keys())
    
    if len(csv_keys) != len(txt_keys):
        print("ERROR not same size")
        return False
    # make sure all in csv are in txt
    for k in csv_keys:
        if k not in txt_keys:
            print('%s not in txt keys' % k)
            return False
    
    # make sure all in txt are in csv
    for k in txt_keys:
        if k not in csv_keys:
            print('%s not in csv keys' % k)
            return False
    return True

def totalCorresponds(csvs):
    train, test, total = csvs
    train = list(train['key'].values)
    test = list(test['key'].values)
    total = list(total['key'].values)
    
    if (len(train) + len(test)) != len(total):
        print("Total is not right length")
        return False
    
    tot_by_v = {}
    for i in total:
        v = i[:5]
        if v not in tot_by_v:
            tot_by_v[v] = []
        tot_by_v[v].append(i)
    
    for i in train + test:
        v = i[:5]
        if i not in tot_by_v[v]:
            print("key in train not in total: %s" % i)
            return False
            
    return True

In [56]:
def sanityCheck(metric):
    print(metric)
    names = ['train', 'test', 'total']
    csvs = getCSV(metric)
    txts = getTXT(metric)
    
    if True:
        for name, csv, txt in zip(names, csvs, txts):
            print()
            print(name)
            print("%s: Same size" % sameSize(csv, txt))
            print("%s: Unique keys" % keysUnique(csv))
            print("%s: All Lower" % allLower(csv, txt))
            #print("%s: Keys Match" % keysMatch(csv, txt))
        
    print("%s: Total corresponds" % totalCorresponds(csvs))
    print()
    print()
    

# Run under here

In [58]:
for metric in ['balanced', 'compo', 'steps']: #  
    sanityCheck(metric)


balanced

train
True: Same size
True: Unique keys
True: All Lower

test
True: Same size
True: Unique keys
True: All Lower

total
True: Same size
True: Unique keys
True: All Lower
True: Total corresponds
compo

train
True: Same size
True: Unique keys
True: All Lower

test
True: Same size
True: Unique keys
True: All Lower

total
True: Same size
True: Unique keys
True: All Lower
True: Total corresponds
steps

train
True: Same size
True: Unique keys
True: All Lower

test
True: Same size
True: Unique keys
True: All Lower

total
True: Same size
True: Unique keys
True: All Lower
True: Total corresponds


In [52]:
end = len(total_steps)
end

1492033

In [53]:
step = int(end / 5)

In [54]:
x = []

for i in range(0, end, step):
    x.append((i, i + step))

In [55]:
x


[(0, 298406),
 (298406, 596812),
 (596812, 895218),
 (895218, 1193624),
 (1193624, 1492030),
 (1492030, 1790436)]

In [40]:
x[10:]

[]

In [59]:
for i in train:
    q = train[i]
    p = q['program']
    if 'forward' in p:
        print(p)
        break

Choose(food, Query(class, OnlyItem(IterateUntil(backward, video, Exists(behind, Filter(frame, [relations])), Filter(frame, [relations, behind, objects])))), IterateUntil(forward, video, XOR(Exists(food, Filter(frame, [relations, touching, objects])), Exists(Query(class, OnlyItem(IterateUntil(backward, video, Exists(behind, Filter(frame, [relations])), Filter(frame, [relations, behind, objects])))), Filter(frame, [relations, touching, objects]))), Filter(frame, [relations, touching, objects])))
